# Runtime MNIST CNN Classification Example
Uses Convs and FCs operations to do a simple. An image scale of [0, 1] is used here.

In [79]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [80]:
BATCH_SIZE = 32
REG_LAMBDA = 0.0001
INITIAL_LR = 0.001
LR_DECAY_STEP_INTERVAL = 10000
LR_DECAY_FACTOR = 0.5

In [81]:
dataset_train = tt.datasets.mnist.MNISTTrainDataset()
dataset_valid = tt.datasets.mnist.MNISTValidDataset()
dataset_test = tt.datasets.mnist.MNISTTestDataset()

### Model

In [82]:
class SimpleClassificationModel(tt.model.AbstractModel):    
    def __init__(self, reg_lambda=0.0):
        super(SimpleClassificationModel, self).__init__(reg_lambda)
        
    @tt.utils.attr.override
    def inference(self, inputs, targets, is_training=True,
                  device_scope=None, memory_device=None):
        # 1: Conv
        conv1 = tt.network.conv2d("Conv1", inputs,
                                  32, (5, 5), (1, 1),
                                  weight_init=tf.contrib.layers.xavier_initializer_conv2d(),
                                  bias_init=0.1,
                                  regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                                  activation=tf.nn.relu)
        conv1 = tt.network.max_pool2d(conv1)

        # 2: Conv
        conv2 = tt.network.conv2d("Conv2", conv1,
                                  64, (3, 3), (1, 1),
                                  weight_init=tf.contrib.layers.xavier_initializer_conv2d(),
                                  bias_init=0.1,
                                  regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                                  activation=tf.nn.relu)
        conv2 = tt.network.max_pool2d(conv2)
        
        conv2_flat = tf.contrib.layers.flatten(conv2)
        
        # 1: FC
        fc1 = tt.network.fc("FC1", conv2_flat, 256,
                            weight_init=tf.contrib.layers.xavier_initializer(),
                            bias_init=0.1,
                            regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                            activation=tf.nn.relu)

        # 2: FC
        fc2 = tt.network.fc("Out", fc1, 10,
                            weight_init=tf.contrib.layers.xavier_initializer(),
                            bias_init=0.1,
                            regularizer=tf.contrib.layers.l2_regularizer(self.reg_lambda),
                            activation=tf.nn.softmax)

        res = tf.reshape(fc2, [-1] + targets.get_shape().as_list()[1:])
        return res
    
    @tt.utils.attr.override
    def loss(self, predictions, targets):
        cross_entropy = -tf.reduce_sum(targets*tf.log(predictions), name="CE_loss")
        return cross_entropy

### Training

In [83]:
runtime = tt.core.DefaultRuntime()
runtime.register_datasets(dataset_train, dataset_valid, dataset_test)
runtime.register_model(SimpleClassificationModel(reg_lambda=REG_LAMBDA))
runtime.build(INITIAL_LR,
              LR_DECAY_STEP_INTERVAL,
              LR_DECAY_FACTOR)

Launing default runtime...
Selecting GPU device: 3
Total model-params: 824970


In [ ]:
runtime.train(batch_size=BATCH_SIZE, steps=1000, do_checkpoints=False, do_summary=False)

### Evaluation

In [ ]:
runtime.test(50)

In [ ]:
x, y = dataset_test.get_batch(1)

tt.visualization.display_array(x[0] * 255)

pred = runtime.predict(x)

print(np.argmax(pred))

### Terminate

In [ ]:
runtime.close()